# County GeoJSON Plotting
State notebook purpose here

### Import Libraries
Import libraries and write settings here.

In [204]:
# Data manipulation
import pandas as pd
import numpy as np
import geopandas as gpd
import os
import folium
from folium.plugins import MarkerCluster
import json
import requests
import ast
import sys
# Options for pandas
pd.options.display.max_columns = None
pd.options.display.max_rows = 30

In [47]:
src_dir = os.path.join(os.getcwd(), '..', '..', 'src')
sys.path.append(src_dir)

In [1]:
# Display all cell outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = 'all'

from IPython import get_ipython
ipython = get_ipython()

# autoreload extension
if 'autoreload' not in ipython.extension_manager.loaded:
    %load_ext autoreload

%autoreload 2

# Visualizations
import plotly.plotly as py
import plotly.graph_objs as go
from plotly.offline import iplot, init_notebook_mode
init_notebook_mode(connected=True)

import cufflinks as cf
cf.go_offline(connected=True)
cf.set_config_file(theme='white')

### Import Data

In [172]:
# Reading in my data: 
data_dropped = pd.read_pickle('../../data/03_processed/county_merged_dropped_NaNs')
data_imp = pd.read_pickle('../../data/03_processed/county_imp_geo')


In [155]:
data_dropped['County FIPS Code'] = data_dropped['County FIPS Code'].apply(lambda x: '0500000US'+str(x))

# Needed this step for the old "data_imp" - handled it when I merged the lat/lon data
#data_imp['County FIPS Code'] = data_imp['County FIPS Code'].apply(lambda x: '0500000US'+str(x))


In [173]:
data_imp.columns

Index(['County FIPS Code', 'County Name', 'Commuting Zone ID',
       'Commuting Zone Name ', 'State', 'Target', 'rank_diff',
       'Number of Children in Core Sample', 'Rank-Rank Slope',
       'Absolute Upward Mobility', 'Top 1% Income Share',
       'Interquartile Income Range', 'Share Between p25 and p75',
       'Mean Parent Income', 'Mean Child Income', 'Parent Income P25',
       'Child Income P25', 'Median Parent Income', 'Median Child Income',
       'Parent Income P75', 'Child Income P75', 'Parent Income P90',
       'Child Income P90', 'Parent Income P99', 'Child Income P99', 'Gini',
       'Teenage Birth Rate', 'Adult obesity', 'Children in poverty',
       'Daily fine particulate matter', 'Diabetic screening',
       'Mammography screening', 'Physical inactivity', 'Premature Death',
       'Preventable hospital stays', 'Sexually transmitted infections',
       'Unemployment', 'Uninsured', 'Violent crime rate', 'lat', 'lon'],
      dtype='object')

In [176]:
data_imp.State

01001    Alabama
01003    Alabama
01005    Alabama
01007    Alabama
01009    Alabama
01011    Alabama
01013    Alabama
01015    Alabama
01017    Alabama
01019    Alabama
01021    Alabama
01023    Alabama
01025    Alabama
01027    Alabama
01029    Alabama
          ...   
56017    Wyoming
56019    Wyoming
56021    Wyoming
56023    Wyoming
56025    Wyoming
56027    Wyoming
56029    Wyoming
56031    Wyoming
56033    Wyoming
56035    Wyoming
56037    Wyoming
56039    Wyoming
56041    Wyoming
56043    Wyoming
56045    Wyoming
Name: State, Length: 3147, dtype: object

## Testing out GeoJSON shape data

Data from:
https://eric.clst.org/tech/usgeojson/

In [15]:
# dc = folium.Map(location=[38.9, -70.03], 
#                 tiles='Mapbox Bright', 
#                 zoom_start=7)
# dc.choropleth('../../data/01_raw/USA/DC.geo.json')

In [150]:
us_county_json = '../../data/01_raw/gz_2010_us_050_00_500k.json'

In [42]:
# with open(us_json, encoding = "ISO-8859-1") as json_file:
#     data = ast.literal_eval(json_file.read())

In [151]:
with open(us_county_json, encoding="ISO-8859-1") as json_file:
    county_json = json.loads(json_file.read())

### Exploring data type/organization to figure out how to filter just DC, MD, VA.

In [152]:
# Testing to see if every GEO_ID is just the actual FIPS Code led by "0500000US" - appears to be the case. 
pd.DataFrame(county_json)['features'][3011]['properties']["GEO_ID"]

# data2['features'][0]['properties']['STATE']

'0500000US54039'

In [83]:
# def my_color_function():
#     """Maps by state"""
#     for i in range(len(data2)):
#         print(f"{i} is {data2['features'][i]['properties']['STATE']}")
#         if data2['features'][i]['properties']['STATE'] == 24:
#             return '#ff0000'
#         else:
#             return '#008000'

The following code works! 

   `m = folium.Map([43, -100], tiles='cartodbpositron', 
                    zoom_start=4)
                    folium.Choropleth(geo_data=data2,
                    fill_color='PuRd',
                    fill_opacity=0.5,
                    line_opacity=0.5,
                    data=data,
                    key_on='properties.GEO_ID',
                    columns=['County FIPS Code', 'Target']).add_to(m)`

Here are some adjustments that were made: 
 - used "geo_data" and instead of the file path, used the opened, decoded file 
 - updated the "data" to have the exact same key as the geojson file 
 - changed fill_color to something that didn't include blue
 - changed "key_on" to exclude "feature." in front of "properties.GEO_ID" (even though the article I was following said that it needed to start with "feature"

In [144]:
# Testing
m = folium.Map([43, -100], tiles='cartodbpositron', zoom_start=4)

folium.Choropleth(geo_data=county_json,
                  fill_color='PuRd',
                  fill_opacity=0.5,
                  line_opacity=0.5,
                  data=data_dropped,
                  key_on='properties.GEO_ID',
                  columns=['County FIPS Code', 'Target']).add_to(m)

In [146]:
m.save(os.path.join('../../results', 'UScountyTEST4.html'))

Yay! The above series of steps worked to FINALLY produce a high-res county outline of the US. 
Key factors:
- dealt with 'utf-8' encoding error using "encoding = "ISO-8859-1""
- instead of using json.load/s - "used ast.literal_eval(json_file.read())" (had to import ast to use that)
    - want to see if it will now work using json.load (used to read in file, while "loads" reads in string). the file is already a streightup series of dictionaries - doesn't even look like it's in a list. And I think folium needs the file as-is...not for it to be read in as a dataframe...but now the question is how to map my actual data...this will be interesting
    - it looks like the encoder I ended up using turns it into a readable format and .read() makes it a string that .loads() is able to parse
- to get the map to show, saved as html file since it wouldn't render quickly (maybe at all in Jupyter notebook)

## Plotting County Data

Use imputed data for mapping (keep more counties)
 1. Displays "Mobile/Not Mobile"
 1. Displays "Absolute Upward Mobility" using graded color scale
 1. 


In [157]:
import branca.colormap as cm

In [161]:
linear = cm.LinearColormap(
    ['red', 'yellow', 'green'],
    vmin=0, vmax=1
)
linear

In [ ]:
bins = list(data_imp['Absolute Upward Mobility'].quantile([0, 0.25, 0.5, 0.75, 1]))

aum_map = folium.Map(location=[48, -102], zoom_start=5)

In [276]:
choro = folium.Choropleth(
    geo_data=county_json,
    data=data_imp,
    columns=['County FIPS Code', 'Absolute Upward Mobility'],
    key_on='properties.GEO_ID',
    fill_color='YlOrRd',
    fill_opacity=0.5,
    nan_fill_color='gray',
    nan_fill_opacity=0.9,
    line_opacity=0.5,
    legend_name='Absolute Upward Mobility',
    bins=bins,
    reset=True, 
    highlight=True,
    name='Absolute Upward Mobility'
    
)

In [284]:
# Showing Target with binary color coding

choro2 = folium.Choropleth(
    geo_data=county_json,
    data=data_imp,
    columns=['County FIPS Code', 'Target'],
    key_on='properties.GEO_ID',
    fill_color='PuBu',
    fill_opacity=0.5,
    nan_fill_color='gray',
    nan_fill_opacity=0.9,
    line_opacity=0.5,
    legend_name='Mobility Outcomes',
    bins=3,
    reset=True, 
    highlight=True,
    name='Child Can Move atleast 1 quartile up'
    )

In [263]:
locations = list(zip(data_imp.lat, data_imp.lon))
popup_content = list(zip(data_imp['County Name'], data_imp['State'], round(data_imp['Absolute Upward Mobility'],1), round(data_imp['Top 1% Income Share']*100, 1)
                         ))
popups = ['<center> {} County, {} <br>  <b>AUM:</b> {} <br><b>Top 1% Income Share:</b> {}% <br>'.format(
    name, state, aum, share) for (name, state, aum, share) in popup_content]

In [289]:
# Testing "Popup without marker when you click the county"

# choro.geojson.add_child(folium.features.GeoJsonTooltip(popups))
# folium.Tooltip()
for idx, row in data_imp.iterrows():
    print(type(idx), idx)
    if row['Absolute Upward Mobility'] >= 50:
        #location = locations[int(idx)][0], locations[int(idx)][1]
        marker = folium.Marker(location=location)    
        popup = popups[idx]
        folium.Popup(popup, max_width='150%').add_to(marker)
#     icons = folium.Icon(color='green', icon='ok-sign').add_to(marker)
        marker.add_to(aum_map)
    else: pass


# choro.add_to(aum_map)
# folium.LayerControl().add_to(aum_map)

<class 'str'> 01001
<class 'str'> 01003
<class 'str'> 01005
<class 'str'> 01007
<class 'str'> 01009
<class 'str'> 01011
<class 'str'> 01013
<class 'str'> 01015
<class 'str'> 01017
<class 'str'> 01019
<class 'str'> 01021
<class 'str'> 01023
<class 'str'> 01025
<class 'str'> 01027
<class 'str'> 01029
<class 'str'> 01031
<class 'str'> 01033
<class 'str'> 01035
<class 'str'> 01037
<class 'str'> 01039
<class 'str'> 01041
<class 'str'> 01043
<class 'str'> 01045
<class 'str'> 01047
<class 'str'> 01049
<class 'str'> 01051
<class 'str'> 01053
<class 'str'> 01055
<class 'str'> 01057
<class 'str'> 01059
<class 'str'> 01061
<class 'str'> 01063
<class 'str'> 01065
<class 'str'> 01067
<class 'str'> 01069
<class 'str'> 01071
<class 'str'> 01073
<class 'str'> 01075
<class 'str'> 01077
<class 'str'> 01079
<class 'str'> 01081
<class 'str'> 01083
<class 'str'> 01085
<class 'str'> 01087
<class 'str'> 01089
<class 'str'> 01091
<class 'str'> 01093
<class 'str'> 01095
<class 'str'> 01097
<class 'str'> 01099


TypeError: list indices must be integers or slices, not str

In [279]:
# choro.geojson.add_child(folium.features.GeoJsonTooltip(['GEO_ID'], aliases=[popups for ]))
choro.add_to(aum_map)
choro2.add_to(aum_map)
folium.LayerControl().add_to(aum_map)

In [192]:
# # Don't want to use the cluster feature on this version
# marker_cluster = MarkerCluster(
#     name='Global Kickstarter Campaigns',
#     overlay=True,
#     control=True,
#     icon_create_function=None
# )

In [213]:
# for k in range(len(data_imp)):
#     if list(data_imp.Target)[k] == 1:
#         location = locations[k][0], locations[k][1]
#         #marker = folium.Marker(location=location)
#         popup = popups[k]
#         folium.Popup(popup, max_width='150%').add_to(aum_map)
#         #icons = folium.Icon(color='green', icon='ok-sign').add_to(marker)
# #         marker_cluster.add_child(marker)
#         #marker.add_to(aum_map)
#     else:
#         pass

# # marker_cluster.add_to(aum_map)

# folium.LayerControl().add_to(aum_map)


In [170]:
#folium.ClickForMarker(popup='Hi').add_to(aum_map)

In [280]:
aum_map.save(os.path.join('../../results', 'aum_map4.html'))

In [166]:
# m_ = folium.Map([43, -100], tiles='cartodbpositron', zoom_start=4)

# folium.GeoJson(
#     county_json,
#     style_function=lambda feature: {
#         'fillColor': linear(data_imp['Absolute Upward Mobility']),
#         'color': 'black',
#         'weight': 2,
#         'dashArray': '5, 5'
#     }
# ).add_to(m_)

# m.save(os.path.join('../../results', 'UScountyTEST5.html'))

# m

## Results
Show graphs and stats here

## Plotly Experiment


In [203]:
import plotly.figure_factory as ff

In [207]:
colorscale = [ "#deebf7", "#d2e3f3", "#c6dbef", "#b3d2e9", "#9ecae1",
    "#85bcdb", "#6baed6", "#57a0ce", "#4292c6", "#3082be", "#2171b5", "#1361a9"
]
endpts = list(np.linspace(.2, .7, len(colorscale) - 1))
fips = data_imp['County FIPS Code'].tolist()
values = data_imp['Absolute Upward Mobility'].tolist()


fig = ff.create_choropleth(
    fips=fips, values=values, scope=['usa'],
    binning_endpoints=endpts, colorscale=colorscale,
    show_state_data=False,
    show_hover=True,
    asp = 2.9,
    title_text = 'AUM',
    legend_title = 'AUM Scale'
)
fig.layout.template = None
fig.show()

[autoreload of geopandas.geoseries failed: Traceback (most recent call last):
  File "/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 244, in check
    superreload(m, reload, self.old_objects)
  File "/anaconda3/lib/python3.7/site-packages/IPython/extensions/autoreload.py", line 378, in superreload
    module = reload(module)
  File "/anaconda3/lib/python3.7/imp.py", line 314, in reload
    return importlib.reload(module)
  File "/anaconda3/lib/python3.7/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 630, in _exec
  File "<frozen importlib._bootstrap_external>", line 728, in exec_module
  File "<frozen importlib._bootstrap>", line 219, in _call_with_frames_removed
  File "/anaconda3/lib/python3.7/site-packages/geopandas/geoseries.py", line 12, in <module>
    from geopandas.base import GeoPandasBase, _series_unary_op, _CoordinateIndexer
ImportError: cannot import name '_series_un

ImportError: geopandas, pyshp and shapely must be installed for this figure factory.

Run the following commands to install the correct versions of the following modules:

```
pip install geopandas==0.3.0
pip install pyshp==1.2.10
pip install shapely==1.6.3
```
If you are using Windows, follow this post to properly install geopandas and dependencies:http://geoffboeing.com/2014/09/using-geopandas-windows/

If you are using Anaconda, do not use PIP to install the packages above. Instead use conda to install them:

```
conda install plotly
conda install geopandas
```

In [ ]:
plotly.offline.plot(aum_map, '../../results/aum_plotly1.html')